<a href="https://colab.research.google.com/github/architvasan/SciFM_Test/blob/main/VirtualScreen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install tensorflow
!pip install transformers
!pip install SmilesPE

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.callbacks import (
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.python.client import device_lib
import json
from smiles_pair_encoders_functions import *
import time

ModuleNotFoundError: No module named 'smiles_pair_encoders_functions'

In [ ]:
with open(json_file) as f:
       params = json.load(f)

In [ ]:
######## Load model #############

model = tf.keras.models.load_model('model.keras')

In [ ]:
vocab_file = 'vocab_spe.txt'
spe_file = 'spe.txt'
tokenizer = SMILES_SPE_Tokenizer(vocab_file = vocab_file, spe_file = spe_file)
maxlen = 45

In [ ]:
def preprocess(data, tokenizer, maxlen):

    tokenized_data = np.array([list(pad(tokenizer(smi)['input_ids'], maxlen, 0)) for smi in data])

    return tokenized_data

In [ ]:
data_smiles = pd.read_csv('input_smiles.smi')

In [ ]:
x_inference = preprocess(data_smiles,
                         tokenizer,
                         maxlen)

In [ ]:
output = model.predict(x_inference, batch_size = 32)